## Still under development

In [1]:
# Agents


In [20]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
# Now you can access the environment variables
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
tavily_api_key= os.getenv('TAVILY_API_KEY')

In [15]:
# Create llm 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=openai_api_key)
# Load document from web
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
# Create the embedder or vectorizer
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=openai_api_key)
# Create the vector store
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
# Create the prompt
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create the chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

# Create the retriever
retriever = vector.as_retriever()

# Create the retrieval chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)


### Start of Agents

In [16]:
# Start of agents
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(TAVILY_API_KEY = tavily_api_key)

In [24]:
tools = [retriever_tool, search]

In [28]:
#%pip install langchainhub
import langchainhub
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [29]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [30]:
# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0,api_key=openai_api_key)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly important for

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in various ways throughout the application development lifecycle. Here are some key ways LangSmith can assist with testing:\n\n1. Prototyping: LangSmith supports rapid experimentation between prompts, model types, retrieval strategies, and other parameters for LLM applications.\n\n2. Debugging: LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues when things go wrong.\n\n3. Initial Test Set: Developers can create datasets with collections of inputs and reference outputs to run tests on LLM applications. LangSmith also allows for custom evaluations to score test results.\n\n4. Comparison View: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application.\n\n5. Playground: LangSmith provides a playground enviro

In [32]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1713032481, 'localtime': '2024-04-13 11:21'}, 'current': {'last_updated_epoch': 1713032100, 'last_updated': '2024-04-13 11:15', 'temp_c': 9.4, 'temp_f': 48.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 15.0, 'wind_kph': 24.1, 'wind_degree': 140, 'wind_dir': 'SE', 'pressure_mb': 1013.0, 'pressure_in': 29.9, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 77, 'cloud': 75, 'feelslike_c': 6.0, 'feelslike_f': 42.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 2.0, 'gust_mph': 21.8, 'gust_kph': 35.0}}"}, {'url': 'https://www.msn.com/en-us/weather/to

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 48.9°F (9.4°C). The wind speed is 24.1 kph coming from the southeast direction. The humidity is at 77%, and the visibility is 9.0 miles.'}

In [3]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

NameError: name 'retriever' is not defined